In [1]:
import os
import glob
import h5py
import numpy as np
import nibabel as nib
from tqdm import tqdm
import matplotlib.pyplot as plt
from ipywidgets import interact,fixed
from utils import normalize_4d_volume,cartesian_mask
from Artifact_simulation import Motion_simulation,Spatial_simulation,Undersampling_simulation,Ghosting_simulation,Spike_simulation,Noise_simulation,Gamma_simulation

In [2]:
ROI_size = [128,128]

seen_fixed_no_of_slices = {}
seen_fixed_no_of_slices["train_support"] = 1500 # less than 1500 for acdc
seen_fixed_no_of_slices["train_query"] = 600 # less than 600 for acdc
seen_fixed_no_of_slices["valid_support"] = 450 # less than 450 for acdc
seen_fixed_no_of_slices["valid_query"] = 500 # less than 500 for acdc

unseen_fixed_no_of_slices = {}
unseen_fixed_no_of_slices["train_support"] = 700
unseen_fixed_no_of_slices["train_query"] = 700
unseen_fixed_no_of_slices["valid_support"] = 700
unseen_fixed_no_of_slices["valid_query"] = 700

## Slice viewer

In [2]:
class slicer():
    
    def __init__(self,np_vol,crop_size = 128):
        
        if np_vol["crp_inp"].dtype == "complex128":
            self.crp_inp = np.abs(np_vol["crp_inp"])
        else:
            self.crp_inp = np_vol["crp_inp"]
        
        self.crp_gt = np_vol["crp_gt"]
        
#         self.vol_dim = np_vol.shape
        
#         self.height_center = int(self.vol_dim[0]/2)
#         self.width_center = int(self.vol_dim[1]/2)
        
#         self.half_crop_size = int(crop_size/2)

    def vol_4d_slice_view(self,slice_view,slice_no_x,slice_no_y,slice_no_t,slice_no_z,fig_size_x,fig_size_y):
        plt.subplots(1,2,figsize=(fig_size_x,fig_size_y))
        if slice_view == 'x':
            plt.subplot(1,2,1)
            plt.imshow(self.crp_inp[slice_no_x,:,:,slice_no_t],cmap='gray')
            plt.title("Input")
            
            plt.subplot(1,2,2)
            plt.imshow(self.crp_gt[slice_no_x,:,:,slice_no_t],cmap='gray')
            plt.title("GT")

        elif slice_view == 'y':
            plt.subplot(1,2,1)
            plt.imshow(self.crp_inp[:,slice_no_y,:,slice_no_t],cmap='gray')
            plt.title("Input")
            
            plt.subplot(1,2,2)
            plt.imshow(self.crp_gt[:,slice_no_y,:,slice_no_t],cmap='gray')
            plt.title("GT")
        
        elif slice_view == 'z':
            plt.subplot(1,2,1)
            plt.imshow(self.crp_inp[:,:,slice_no_z,slice_no_t],cmap='gray')
            plt.title("Input")
            
            plt.subplot(1,2,2)
            plt.imshow(self.crp_gt[:,:,slice_no_z,slice_no_t],cmap='gray')
            plt.title("GT")

        elif slice_view == 't':
            plt.subplot(1,2,1)
            plt.imshow(self.crp_inp[:,:,slice_no_z,slice_no_t],cmap='gray')
            plt.title("Input")
            
            plt.subplot(1,2,2)
            plt.imshow(self.crp_gt[:,:,slice_no_z,slice_no_t],cmap='gray')
            plt.title("GT")
        
    def vol_3d_slice_view(self,vol,slice_view,slice_no_x,slice_no_y,slice_no_z,fig_size_x,fig_size_y):
        plt.figure(figsize=(fig_size_x,fig_size_y))
        if slice_view == 'x':
            plt.imshow(vol[slice_no_x,:,:],cmap='gray')
            plt.show()
        elif slice_view == 'y':
            plt.imshow(vol[:,slice_no_y,:],cmap='gray')
            plt.show()

        elif slice_view == 'z':
            plt.imshow(vol[:,:,slice_no_z],cmap='gray')
            plt.show()
            
    def slicer_view(self):
        slice_no_x_max = self.crp_gt.shape[0]-1
        slice_no_y_max = self.crp_gt.shape[1]-1
        slice_no_z_max = self.crp_gt.shape[2]-1
        slice_no_t_max = self.crp_gt.shape[3]-1
        return interact(self.vol_4d_slice_view,slice_view = ['z','t','x','y'],slice_no_x = (0,slice_no_x_max,1),slice_no_y = (0,slice_no_y_max,1),slice_no_z = (0,slice_no_z_max,1),slice_no_t = (0,slice_no_t_max,1),fig_size_x=(5,8),fig_size_y=(5,8))  

In [3]:
disp_file_name = glob.glob("/media/Data/MRI/datasets/artifact_suppression/Motion/05/train_support/*.h5")[0]

f = h5py.File(disp_file_name,'r')

slice_viewer = slicer(f)
slice_viewer.slicer_view()

interactive(children=(Dropdown(description='slice_view', options=('z', 't', 'x', 'y'), value='z'), IntSlider(v…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [ ]:
f.close()

# ACDC

## Training tasks --- Motion Artifact simulation

In [3]:
src_path = "/home/arun/datasets/ACDC_training/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/artifact_suppression/"

degradation_type = "Motion" # Change according to the artifact to be simulated

no_of_train_spt = 50 # Always less than 50 # 25 for M/03, 36 for M/05, 50 for M/07 --- all tasks train support data should have 1500 slices each
no_of_train_qry = 20 # Always less than 20 # 8 for M/03, 17 for M/05, 20 for M/07 --- all tasks train query data should have 600 slices each

no_of_val_spt = 15 # Always less than 15 # 6 for M/03, 12 for M/05, 15 for M/07 --- all tasks validation support data should have 450 slices each
no_of_val_qry = 15 # Always less than 15 # 10 for M/03, 13 for M/05, 15 for M/07 --- all tasks validation query data should have 500 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [5]:
no_of_frame_mixes = [3,5,7]
for frame_mix_number in no_of_frame_mixes:
    for number_of_slices_key,i in tqdm(zip(seen_fixed_no_of_slices,dict_file_names)):

        folder_bits = i.split("_")
        folder_cat = degradation_type+"/"+"0{}/".format(frame_mix_number)+folder_bits[0]+"_"+folder_bits[1]+"/"
        clear_dst_dir = dst_dir+folder_cat

        if not os.path.exists(clear_dst_dir):
            os.makedirs(clear_dst_dir)

        specific_folder_file_names = dict_file_names[i]

        count_slices = 0
        for file_name in specific_folder_file_names:

            with h5py.File(file_name,'r') as hf1:

                img_vol = hf1['inpVol']

                x_center = hf1["x_center"][0]
                y_center = hf1["y_center"][0]

                norm_img_vol = normalize_4d_volume(img_vol)

                clean_4d_vol,corrupted_4d_vol = Motion_simulation(norm_img_vol,frame_mix_number)

                x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
                if np.min(x_coords) < 0:
                    x_coords = x_coords-np.min(x_coords)

                y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
                if np.min(y_coords) < 0:
                    y_coords = y_coords-np.min(y_coords)

                corrupted_cropped_4d_vol = corrupted_4d_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                cropped_gt_vol = clean_4d_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

                patient_name = file_name.split("/")[-1].split("_")[0]
                file_final_destination = clear_dst_dir+patient_name+".h5"

                with h5py.File(file_final_destination,'w') as hf2:

                    hf2.create_dataset('gt',data = clean_4d_vol)

                    hf2.create_dataset("inp",data = corrupted_4d_vol)

                    hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                    hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                    hf2.create_dataset("x_center",data = [x_center])

                    hf2.create_dataset("y_center",data = [y_center])

            if count_slices >= seen_fixed_no_of_slices[number_of_slices_key]:
                break


In [12]:
disp_file_name = glob.glob("/media/Data/MRI/datasets/artifact_suppression/Motion/07/train_support/*.h5")[0]

f = h5py.File(disp_file_name,'r')

slice_viewer = slicer(f)
slice_viewer.slicer_view()

interactive(children=(Dropdown(description='slice_view', options=('z', 't', 'x', 'y'), value='z'), IntSlider(v…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [10]:
f.close()

## Unseen tasks --- Motion Artifact simulation

In [3]:
src_path = "/home/arun/datasets/ACDC_training/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/unseen_artifact_suppression/"

degradation_type = "Motion" # Change according to the artifact to be simulated

no_of_train_spt = 25 # 15 for M/04, 22 for M/06 --- all tasks train support data should have 700 slices each
no_of_train_qry = 25 # 12 for M/04, 18 for M/06 --- all tasks train query data should have 700 slices each

no_of_val_spt = 25 # 11 for M/04, 23 for M/06 --- all tasks validation support data should have 700 slices each
no_of_val_qry = 25 # 16 for M/04, 23 for M/06 --- all tasks validation query data should have 700 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [4]:
no_of_frame_mixes = [4,6]
for frame_mix_number in no_of_frame_mixes:
    for number_of_slices_key,i in tqdm(zip(unseen_fixed_no_of_slices,dict_file_names)):

        folder_bits = i.split("_")
        folder_cat = degradation_type+"/"+"0{}/".format(frame_mix_number)+folder_bits[0]+"_"+folder_bits[1]+"/"
        clear_dst_dir = dst_dir+folder_cat

        if not os.path.exists(clear_dst_dir):
            os.makedirs(clear_dst_dir)

        specific_folder_file_names = dict_file_names[i]

        count_slices = 0
        for file_name in specific_folder_file_names:

            with h5py.File(file_name,'r') as hf1:

                img_vol = hf1['inpVol']

                x_center = hf1["x_center"][0]
                y_center = hf1["y_center"][0]

                norm_img_vol = normalize_4d_volume(img_vol)

                clean_4d_vol,corrupted_4d_vol = Motion_simulation(norm_img_vol,frame_mix_number)

                x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
                if np.min(x_coords) < 0:
                    x_coords = x_coords-np.min(x_coords)

                y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
                if np.min(y_coords) < 0:
                    y_coords = y_coords-np.min(y_coords)

                corrupted_cropped_4d_vol = corrupted_4d_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                cropped_gt_vol = clean_4d_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

                patient_name = file_name.split("/")[-1].split("_")[0]
                file_final_destination = clear_dst_dir+patient_name+".h5"

                with h5py.File(file_final_destination,'w') as hf2:

                    hf2.create_dataset('gt',data = clean_4d_vol)

                    hf2.create_dataset("inp",data = corrupted_4d_vol)

                    hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                    hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                    hf2.create_dataset("x_center",data = [x_center])

                    hf2.create_dataset("y_center",data = [y_center])

            if count_slices >= unseen_fixed_no_of_slices[number_of_slices_key]:
                break


0it [00:00, ?it/s]/home/arun/dataset_creation/Artifact_simulation.py:37: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  ksp_line_seq = np.hstack(([linenum for linenum in range(lineblock,rows, n_by_etl)] for lineblock in range(0,n_by_etl)))
4it [08:16, 124.17s/it]


In [13]:
disp_file_name = glob.glob("/media/Data/MRI/datasets/unseen_artifact_suppression/Motion/04/train_support/*.h5")[0]

f = h5py.File(disp_file_name,'r')

slice_viewer = slicer(f)
slice_viewer.slicer_view()

interactive(children=(Dropdown(description='slice_view', options=('z', 't', 'x', 'y'), value='z'), IntSlider(v…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [14]:
f.close()

## Training tasks --- Spatial Artifact simulation

In [75]:
src_path = "/home/arun/datasets/ACDC_training/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/artifact_suppression/"

degradation_type = "Spatial" # Change according to the artifact to be simulated

no_of_train_spt = 50 # 9 --- all tasks train support data should have 1500 slices each
no_of_train_qry = 20 # 3 --- all tasks train query data should have 600 slices each

no_of_val_spt = 15 # 3 --- all tasks validation support data should have 450 slices each
no_of_val_qry = 15 # 3 --- all tasks validation query data should have 500 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [ ]:
downsampling_factors = [2,3,5]

for downsampling_factor in downsampling_factors:
    for number_of_slices_key,i in tqdm(zip(seen_fixed_no_of_slices,dict_file_names)):

        folder_bits = i.split("_")
        folder_cat = degradation_type+"/"+"0{}/".format(downsampling_factor)+folder_bits[0]+"_"+folder_bits[1]+"/"
        clear_dst_dir = dst_dir+folder_cat

        if not os.path.exists(clear_dst_dir):
            os.makedirs(clear_dst_dir)

        specific_folder_file_names = dict_file_names[i]

        count_slices = 0
        for file_name in specific_folder_file_names:

            with h5py.File(file_name,'r') as hf1:

                img_vol = hf1['inpVol']

                x_center = hf1["x_center"][0]
                y_center = hf1["y_center"][0]

                norm_img_vol = normalize_4d_volume(img_vol)

                corrupted_4d_vol = Spatial_simulation(norm_img_vol,downsampling_factor)

                x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
                if np.min(x_coords) < 0:
                    x_coords = x_coords-np.min(x_coords)

                y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
                if np.min(y_coords) < 0:
                    y_coords = y_coords-np.min(y_coords)

                corrupted_cropped_4d_vol = corrupted_4d_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                cropped_gt_vol = norm_img_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

                patient_name = file_name.split("/")[-1].split("_")[0]
                file_final_destination = clear_dst_dir+patient_name+".h5"

                with h5py.File(file_final_destination,'w') as hf2:

                    hf2.create_dataset('gt',data = norm_img_vol)

                    hf2.create_dataset("inp",data = corrupted_4d_vol)

                    hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                    hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                    hf2.create_dataset("x_center",data = [x_center])

                    hf2.create_dataset("y_center",data = [y_center])

            if count_slices >= seen_fixed_no_of_slices[number_of_slices_key]:
                break


In [11]:
disp_file_name = glob.glob("/media/Data/MRI/datasets/artifact_suppression/Spatial/05/train_support/*.h5")[0]

f = h5py.File(disp_file_name,'r')

slice_viewer = slicer(f)
slice_viewer.slicer_view()

interactive(children=(Dropdown(description='slice_view', options=('z', 't', 'x', 'y'), value='z'), IntSlider(v…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [18]:
f.close()

## Unseen tasks --- Spatial Artifact simulation

In [63]:
src_path = "/home/arun/datasets/ACDC_training/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/unseen_artifact_suppression/"

degradation_type = "Spatial" # Change according to the artifact to be simulated

no_of_train_spt = 25 # 5 --- all tasks train support data should have 700 slices each
no_of_train_qry = 25 # 4 --- all tasks train query data should have 700 slices each

no_of_val_spt = 25 # 4 --- all tasks validation support data should have 700 slices each
no_of_val_qry = 25 # 4 --- all tasks validation query data should have 700 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [64]:
downsampling_factors = [4]

for downsampling_factor in downsampling_factors:
    for number_of_slices_key,i in tqdm(zip(unseen_fixed_no_of_slices,dict_file_names)):

        folder_bits = i.split("_")
        folder_cat = degradation_type+"/"+"0{}/".format(downsampling_factor)+folder_bits[0]+"_"+folder_bits[1]+"/"
        clear_dst_dir = dst_dir+folder_cat

        if not os.path.exists(clear_dst_dir):
            os.makedirs(clear_dst_dir)

        specific_folder_file_names = dict_file_names[i]

        count_slices = 0
        for file_name in specific_folder_file_names:

            with h5py.File(file_name,'r') as hf1:

                img_vol = hf1['inpVol']

                x_center = hf1["x_center"][0]
                y_center = hf1["y_center"][0]

                norm_img_vol = normalize_4d_volume(img_vol)

                corrupted_4d_vol = Spatial_simulation(norm_img_vol,downsampling_factor)

                x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
                if np.min(x_coords) < 0:
                    x_coords = x_coords-np.min(x_coords)

                y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
                if np.min(y_coords) < 0:
                    y_coords = y_coords-np.min(y_coords)

                corrupted_cropped_4d_vol = corrupted_4d_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                cropped_gt_vol = norm_img_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

                patient_name = file_name.split("/")[-1].split("_")[0]
                file_final_destination = clear_dst_dir+patient_name+".h5"

                with h5py.File(file_final_destination,'w') as hf2:

                    hf2.create_dataset('gt',data = norm_img_vol)

                    hf2.create_dataset("inp",data = corrupted_4d_vol)

                    hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                    hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                    hf2.create_dataset("x_center",data = [x_center])

                    hf2.create_dataset("y_center",data = [y_center])

            if count_slices >= unseen_fixed_no_of_slices[number_of_slices_key]:
                break


100%|█████████████████████████████████████████████| 1/1 [00:23<00:00, 23.13s/it]


In [7]:
disp_file_name = glob.glob("/media/Data/MRI/datasets/unseen_artifact_suppression/Spatial/04/train_support/*.h5")[0]

f = h5py.File(disp_file_name,'r')

slice_viewer = slicer(f)
slice_viewer.slicer_view()

interactive(children=(Dropdown(description='slice_view', options=('z', 't', 'x', 'y'), value='z'), IntSlider(v…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [ ]:
f.close()

## Training tasks --- Undersampling Artifact simulation

In [3]:
src_path = "/home/arun/datasets/ACDC_training/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/artifact_suppression/"

degradation_type = "Undersampling" # Change according to the artifact to be simulated

no_of_train_spt = 50 # 9 --- all tasks train support data should have 1500 slices each
no_of_train_qry = 20 # 3 --- all tasks train query data should have 600 slices each

no_of_val_spt = 15 # 3 --- all tasks validation support data should have 450 slices each
no_of_val_qry = 15 # 3 --- all tasks validation query data should have 500 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [4]:
us_factors = [2,4,6]
sample_n = 12 ## number of lines in center 

for us_factor in us_factors:
    for number_of_slices_key,i in tqdm(zip(seen_fixed_no_of_slices,dict_file_names)):

        folder_bits = i.split("_")
        folder_cat = degradation_type+"/"+"0{}/".format(us_factor)+folder_bits[0]+"_"+folder_bits[1]+"/"
        clear_dst_dir = dst_dir+folder_cat
        
        us_mask_path = "/home/arun/datasets/npy_masks/mask_0{}.npy".format(us_factor)
        if not os.path.exists(us_mask_path):
            npy_mask = cartesian_mask(ROI_size,us_factor,sample_n,centred=False)
            np.save(us_mask_path,npy_mask) 

        if not os.path.exists(clear_dst_dir):
            os.makedirs(clear_dst_dir)

        specific_folder_file_names = dict_file_names[i]

        count_slices = 0
        for file_name in specific_folder_file_names:

            with h5py.File(file_name,'r') as hf1:

                img_vol = hf1['inpVol']

                x_center = hf1["x_center"][0]
                y_center = hf1["y_center"][0]

                norm_img_vol = normalize_4d_volume(img_vol)

                x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
                if np.min(x_coords) < 0:
                    x_coords = x_coords-np.min(x_coords)

                y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
                if np.min(y_coords) < 0:
                    y_coords = y_coords-np.min(y_coords)

                cropped_gt_vol = norm_img_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]
                
                height,width,no_of_slices,no_of_frames = cropped_gt_vol.shape
                corrupted_cropped_4d_vol = np.zeros(cropped_gt_vol.shape)
                corrupted_cropped_4d_ksp_vol = np.zeros(cropped_gt_vol.shape)
                for frame_no in range(no_of_frames):

                    corrupted_cropped_4d_vol[:,:,:,frame_no],corrupted_cropped_4d_ksp_vol[:,:,:,frame_no] = Undersampling_simulation(cropped_gt_vol[:,:,:,frame_no],us_factor,us_mask_path)

                count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

                patient_name = file_name.split("/")[-1].split("_")[0]
                file_final_destination = clear_dst_dir+patient_name+".h5"

                with h5py.File(file_final_destination,'w') as hf2:

                    hf2.create_dataset('gt',data = norm_img_vol)

                    # hf2.create_dataset("inp",data = corrupted_4d_vol)

                    hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                    hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                    hf2.create_dataset("crp_ksp",data = corrupted_cropped_4d_ksp_vol)

                    hf2.create_dataset("ksp_mask",data = np.load(us_mask_path))

                    hf2.create_dataset("x_center",data = [x_center])

                    hf2.create_dataset("y_center",data = [y_center])

            if count_slices >= seen_fixed_no_of_slices[number_of_slices_key]:
                break


100%|█████████████████████████████████████████████| 4/4 [00:30<00:00,  7.68s/it]


In [22]:
disp_file_name = glob.glob("/media/Data/MRI/datasets/artifact_suppression/Undersampling/07/train_support/*.h5")[0]

f = h5py.File(disp_file_name,'r')

slice_viewer = slicer(f)
slice_viewer.slicer_view()

interactive(children=(Dropdown(description='slice_view', options=('z', 't', 'x', 'y'), value='z'), IntSlider(v…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [23]:
f.close()

## Unseen tasks --- Undersampling Artifact simulation

In [3]:
src_path = "/home/arun/datasets/ACDC_training/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/unseen_artifact_suppression/"

degradation_type = "Undersampling" # Change according to the artifact to be simulated

no_of_train_spt = 25 # 5 --- all tasks train support data should have 700 slices each
no_of_train_qry = 25 # 4 --- all tasks train query data should have 700 slices each

no_of_val_spt = 25 # 4 --- all tasks validation support data should have 700 slices each
no_of_val_qry = 25 # 4 --- all tasks validation query data should have 700 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [4]:
us_factors = [3,5]
sample_n = 12 ## number of lines in center 

for us_factor in us_factors:
    for number_of_slices_key,i in tqdm(zip(unseen_fixed_no_of_slices,dict_file_names)):

        folder_bits = i.split("_")
        folder_cat = degradation_type+"/"+"0{}/".format(us_factor)+folder_bits[0]+"_"+folder_bits[1]+"/"
        clear_dst_dir = dst_dir+folder_cat
        
        us_mask_path = "/home/arun/datasets/npy_masks/mask_0{}.npy".format(us_factor)
        if not os.path.exists(us_mask_path):
            npy_mask = cartesian_mask(ROI_size,us_factor,sample_n,centred=False)
            np.save(us_mask_path,npy_mask) 

        if not os.path.exists(clear_dst_dir):
            os.makedirs(clear_dst_dir)

        specific_folder_file_names = dict_file_names[i]

        count_slices = 0
        for file_name in specific_folder_file_names:

            with h5py.File(file_name,'r') as hf1:

                img_vol = hf1['inpVol']

                x_center = hf1["x_center"][0]
                y_center = hf1["y_center"][0]

                norm_img_vol = normalize_4d_volume(img_vol)

                x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
                if np.min(x_coords) < 0:
                    x_coords = x_coords-np.min(x_coords)

                y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
                if np.min(y_coords) < 0:
                    y_coords = y_coords-np.min(y_coords)

                cropped_gt_vol = norm_img_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]
                
                height,width,no_of_slices,no_of_frames = cropped_gt_vol.shape
                corrupted_cropped_4d_vol = np.zeros(cropped_gt_vol.shape)
                corrupted_cropped_4d_ksp_vol = np.zeros(cropped_gt_vol.shape)
                for frame_no in range(no_of_frames):

                    corrupted_cropped_4d_vol[:,:,:,frame_no],corrupted_cropped_4d_ksp_vol[:,:,:,frame_no] = Undersampling_simulation(cropped_gt_vol[:,:,:,frame_no],us_factor,us_mask_path)

                count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

                patient_name = file_name.split("/")[-1].split("_")[0]
                file_final_destination = clear_dst_dir+patient_name+".h5"

                with h5py.File(file_final_destination,'w') as hf2:

                    hf2.create_dataset('gt',data = norm_img_vol)

                    # hf2.create_dataset("inp",data = corrupted_4d_vol)

                    hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                    hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                    hf2.create_dataset("crp_ksp",data = corrupted_cropped_4d_ksp_vol)

                    hf2.create_dataset("ksp_mask",data = np.load(us_mask_path))

                    hf2.create_dataset("x_center",data = [x_center])

                    hf2.create_dataset("y_center",data = [y_center])

            if count_slices >= unseen_fixed_no_of_slices[number_of_slices_key]:
                break


0it [00:00, ?it/s]/tmp/ipykernel_2494640/3714984296.py:48: ComplexWarning: Casting complex values to real discards the imaginary part
  corrupted_cropped_4d_vol[:,:,:,frame_no],corrupted_cropped_4d_ksp_vol[:,:,:,frame_no] = Undersampling_simulation(cropped_gt_vol[:,:,:,frame_no],us_factor,us_mask_path)
4it [01:38, 24.73s/it]
4it [01:38, 24.54s/it]


In [ ]:
disp_file_name = glob.glob("/media/Data/MRI/datasets/artifact_suppression/Undersampling/08/train_support/*.h5")[0]

f = h5py.File(disp_file_name,'r')

slice_viewer = slicer(f)
slice_viewer.slicer_view()

In [ ]:
f.close()

## Unseen tasks --- Ghosting Artifact simulation

In [3]:
src_path = "/home/arun/datasets/ACDC_training/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/unseen_artifact_suppression/"

degradation_type = "Ghosting" # Change according to the artifact to be simulated

no_of_train_spt = 25 # 5 --- all tasks train support data should have 700 slices each
no_of_train_qry = 25 # 4 --- all tasks train query data should have 700 slices each

no_of_val_spt = 25 # 4 --- all tasks validation support data should have 700 slices each
no_of_val_qry = 25 # 4 --- all tasks validation query data should have 700 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [4]:
num_ghosts = [4,6]

for num_ghost in num_ghosts:
    for number_of_slices_key,i in tqdm(zip(unseen_fixed_no_of_slices,dict_file_names)):

        folder_bits = i.split("_")
        folder_cat = degradation_type+"/"+"0{}/".format(num_ghost)+folder_bits[0]+"_"+folder_bits[1]+"/"
        clear_dst_dir = dst_dir+folder_cat

        if not os.path.exists(clear_dst_dir):
            os.makedirs(clear_dst_dir)

        specific_folder_file_names = dict_file_names[i]

        count_slices = 0
        for file_name in specific_folder_file_names:

            with h5py.File(file_name,'r') as hf1:

                img_vol = hf1['inpVol']

                x_center = hf1["x_center"][0]
                y_center = hf1["y_center"][0]

                norm_img_vol = normalize_4d_volume(img_vol)

                # corrupted_4d_vol = Ghosting_simulation(norm_img_vol,num_ghost)

                x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
                if np.min(x_coords) < 0:
                    x_coords = x_coords-np.min(x_coords)

                y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
                if np.min(y_coords) < 0:
                    y_coords = y_coords-np.min(y_coords)

                # corrupted_cropped_4d_vol = corrupted_4d_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                cropped_gt_vol = norm_img_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                corrupted_cropped_4d_vol = Ghosting_simulation(cropped_gt_vol,num_ghost)
                corrupted_cropped_4d_vol = np.float64(corrupted_cropped_4d_vol)

                count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

                patient_name = file_name.split("/")[-1].split("_")[0]
                file_final_destination = clear_dst_dir+patient_name+".h5"

                with h5py.File(file_final_destination,'w') as hf2:

                    hf2.create_dataset('gt',data = norm_img_vol)

                    # hf2.create_dataset("inp",data = corrupted_4d_vol)

                    hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                    hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                    hf2.create_dataset("x_center",data = [x_center])

                    hf2.create_dataset("y_center",data = [y_center])

            if count_slices >= unseen_fixed_no_of_slices[number_of_slices_key]:
                break


4it [01:47, 26.82s/it]
4it [01:46, 26.55s/it]


## Unseen tasks --- Spike Artifact simulation

In [3]:
src_path = "/home/arun/datasets/ACDC_training/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/unseen_artifact_suppression/"

degradation_type = "Spiking" # Change according to the artifact to be simulated

no_of_train_spt = 25 # 5 --- all tasks train support data should have 700 slices each
no_of_train_qry = 25 # 4 --- all tasks train query data should have 700 slices each

no_of_val_spt = 25 # 4 --- all tasks validation support data should have 700 slices each
no_of_val_qry = 25 # 4 --- all tasks validation query data should have 700 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [4]:
num_spikes = [3,5]

for num_spike in num_spikes:
    for number_of_slices_key,i in tqdm(zip(unseen_fixed_no_of_slices,dict_file_names)):

        folder_bits = i.split("_")
        folder_cat = degradation_type+"/"+"0{}/".format(num_spike)+folder_bits[0]+"_"+folder_bits[1]+"/"
        clear_dst_dir = dst_dir+folder_cat

        if not os.path.exists(clear_dst_dir):
            os.makedirs(clear_dst_dir)

        specific_folder_file_names = dict_file_names[i]

        count_slices = 0
        for file_name in specific_folder_file_names:

            with h5py.File(file_name,'r') as hf1:

                img_vol = hf1['inpVol']

                x_center = hf1["x_center"][0]
                y_center = hf1["y_center"][0]

                norm_img_vol = normalize_4d_volume(img_vol)

                # corrupted_4d_vol = Spike_simulation(norm_img_vol,num_spike)

                x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
                if np.min(x_coords) < 0:
                    x_coords = x_coords-np.min(x_coords)

                y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
                if np.min(y_coords) < 0:
                    y_coords = y_coords-np.min(y_coords)

                # corrupted_cropped_4d_vol = corrupted_4d_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                cropped_gt_vol = norm_img_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                corrupted_cropped_4d_vol = Spike_simulation(cropped_gt_vol,num_spike)
                corrupted_cropped_4d_vol = np.float64(corrupted_cropped_4d_vol)

                count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

                patient_name = file_name.split("/")[-1].split("_")[0]
                file_final_destination = clear_dst_dir+patient_name+".h5"

                with h5py.File(file_final_destination,'w') as hf2:

                    hf2.create_dataset('gt',data = norm_img_vol)

                    # hf2.create_dataset("inp",data = corrupted_4d_vol)

                    hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                    hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                    hf2.create_dataset("x_center",data = [x_center])

                    hf2.create_dataset("y_center",data = [y_center])

            if count_slices >= unseen_fixed_no_of_slices[number_of_slices_key]:
                break


4it [01:46, 26.58s/it]
4it [01:46, 26.74s/it]


## Unseen tasks --- Noise Artifact simulation

In [3]:
src_path = "/home/arun/datasets/ACDC_training/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/unseen_artifact_suppression/"

degradation_type = "Noise" # Change according to the artifact to be simulated

no_of_train_spt = 25 # 5 --- all tasks train support data should have 700 slices each
no_of_train_qry = 25 # 4 --- all tasks train query data should have 700 slices each

no_of_val_spt = 25 # 4 --- all tasks validation support data should have 700 slices each
no_of_val_qry = 25 # 4 --- all tasks validation query data should have 700 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [4]:
stds = [3,4]

for std in stds:
    for number_of_slices_key,i in tqdm(zip(unseen_fixed_no_of_slices,dict_file_names)):

        folder_bits = i.split("_")
        folder_cat = degradation_type+"/"+"0{}/".format(std)+folder_bits[0]+"_"+folder_bits[1]+"/"
        clear_dst_dir = dst_dir+folder_cat

        if not os.path.exists(clear_dst_dir):
            os.makedirs(clear_dst_dir)

        specific_folder_file_names = dict_file_names[i]

        count_slices = 0
        for file_name in specific_folder_file_names:

            with h5py.File(file_name,'r') as hf1:

                img_vol = hf1['inpVol']

                x_center = hf1["x_center"][0]
                y_center = hf1["y_center"][0]

                norm_img_vol = normalize_4d_volume(img_vol)

                # corrupted_4d_vol = Noise_simulation(norm_img_vol,std = std/100)

                x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
                if np.min(x_coords) < 0:
                    x_coords = x_coords-np.min(x_coords)

                y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
                if np.min(y_coords) < 0:
                    y_coords = y_coords-np.min(y_coords)

                # corrupted_cropped_4d_vol = corrupted_4d_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                cropped_gt_vol = norm_img_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                corrupted_cropped_4d_vol = Noise_simulation(cropped_gt_vol,std = std/100)
                corrupted_cropped_4d_vol = np.float64(corrupted_cropped_4d_vol)

                count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

                patient_name = file_name.split("/")[-1].split("_")[0]
                file_final_destination = clear_dst_dir+patient_name+".h5"

                with h5py.File(file_final_destination,'w') as hf2:

                    hf2.create_dataset('gt',data = norm_img_vol)

                    # hf2.create_dataset("inp",data = corrupted_4d_vol)

                    hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                    hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                    hf2.create_dataset("x_center",data = [x_center])

                    hf2.create_dataset("y_center",data = [y_center])

            if count_slices >= unseen_fixed_no_of_slices[number_of_slices_key]:
                break


4it [01:43, 25.89s/it]
4it [01:43, 25.86s/it]


## Unseen tasks --- Gamma Artifact simulation

In [3]:
src_path = "/home/arun/datasets/ACDC_training/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/unseen_artifact_suppression/"

degradation_type = "Gamma" # Change according to the artifact to be simulated

no_of_train_spt = 25 # 5 --- all tasks train support data should have 700 slices each
no_of_train_qry = 25 # 4 --- all tasks train query data should have 700 slices each

no_of_val_spt = 25 # 4 --- all tasks validation support data should have 700 slices each
no_of_val_qry = 25 # 4 --- all tasks validation query data should have 700 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [4]:
gammas = [1,2]

for gamma in gammas:
    for number_of_slices_key,i in tqdm(zip(unseen_fixed_no_of_slices,dict_file_names)):

        folder_bits = i.split("_")
        folder_cat = degradation_type+"/"+"0{}/".format(gamma)+folder_bits[0]+"_"+folder_bits[1]+"/"
        clear_dst_dir = dst_dir+folder_cat

        if not os.path.exists(clear_dst_dir):
            os.makedirs(clear_dst_dir)

        specific_folder_file_names = dict_file_names[i]

        count_slices = 0
        for file_name in specific_folder_file_names:

            with h5py.File(file_name,'r') as hf1:

                img_vol = hf1['inpVol']

                x_center = hf1["x_center"][0]
                y_center = hf1["y_center"][0]

                norm_img_vol = normalize_4d_volume(img_vol)

                # corrupted_4d_vol = Gamma_simulation(norm_img_vol,-gamma/10)
                # corrupted_4d_vol = np.float64(corrupted_4d_vol)

                x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
                if np.min(x_coords) < 0:
                    x_coords = x_coords-np.min(x_coords)

                y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
                if np.min(y_coords) < 0:
                    y_coords = y_coords-np.min(y_coords)

                # corrupted_cropped_4d_vol = corrupted_4d_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                cropped_gt_vol = norm_img_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                corrupted_cropped_4d_vol = Gamma_simulation(cropped_gt_vol,-gamma/10)
                corrupted_cropped_4d_vol = np.float64(corrupted_cropped_4d_vol)

                count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

                patient_name = file_name.split("/")[-1].split("_")[0]
                file_final_destination = clear_dst_dir+patient_name+".h5"

                with h5py.File(file_final_destination,'w') as hf2:

                    hf2.create_dataset('gt',data = norm_img_vol)

                    # hf2.create_dataset("inp",data = corrupted_4d_vol)

                    hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                    hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                    hf2.create_dataset("x_center",data = [x_center])

                    hf2.create_dataset("y_center",data = [y_center])

            if count_slices >= unseen_fixed_no_of_slices[number_of_slices_key]:
                break


4it [01:36, 24.11s/it]
4it [01:36, 24.14s/it]


# M&M

## Training tasks --- Motion Artifact simulation M&M

In [3]:
src_path = "/home/arun/datasets/M_M/Training/Labeled/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/MM_artifact_suppression/"

degradation_type = "MotionM" # Change according to the artifact to be simulated

no_of_train_spt = 70 # --- all tasks train support data should have 1500 slices each
no_of_train_qry = 40 # --- all tasks train query data should have 600 slices each

no_of_val_spt = 20 # --- all tasks validation support data should have 450 slices each
no_of_val_qry = 20 # --- all tasks validation query data should have 500 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [5]:
no_of_frame_mixes = [3,5,7]
for frame_mix_number in no_of_frame_mixes:
    for number_of_slices_key,i in tqdm(zip(seen_fixed_no_of_slices,dict_file_names)):

        folder_bits = i.split("_")
        folder_cat = degradation_type+"/"+"0{}/".format(frame_mix_number)+folder_bits[0]+"_"+folder_bits[1]+"/"
        clear_dst_dir = dst_dir+folder_cat

        if not os.path.exists(clear_dst_dir):
            os.makedirs(clear_dst_dir)

        specific_folder_file_names = dict_file_names[i]

        count_slices = 0
        for file_name in specific_folder_file_names:

            with h5py.File(file_name,'r') as hf1:

                img_vol = hf1['inpVol']

                x_center = hf1["x_center"][0]
                y_center = hf1["y_center"][0]

                norm_img_vol = normalize_4d_volume(img_vol)

                clean_4d_vol,corrupted_4d_vol = Motion_simulation(norm_img_vol,frame_mix_number)

                x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
                if np.min(x_coords) < 0:
                    x_coords = x_coords-np.min(x_coords)

                y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
                if np.min(y_coords) < 0:
                    y_coords = y_coords-np.min(y_coords)

                corrupted_cropped_4d_vol = corrupted_4d_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                cropped_gt_vol = clean_4d_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

                patient_name = file_name.split("/")[-1].split("_")[0]
                file_final_destination = clear_dst_dir+patient_name+".h5"

                with h5py.File(file_final_destination,'w') as hf2:

                    hf2.create_dataset('gt',data = clean_4d_vol)

                    hf2.create_dataset("inp",data = corrupted_4d_vol)

                    hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                    hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                    hf2.create_dataset("x_center",data = [x_center])

                    hf2.create_dataset("y_center",data = [y_center])

            if count_slices >= seen_fixed_no_of_slices[number_of_slices_key]:
                break


In [12]:
disp_file_name = glob.glob("/media/Data/MRI/datasets/M&M_artifact_suppression/MotionM/07/train_support/*.h5")[0]

f = h5py.File(disp_file_name,'r')

slice_viewer = slicer(f)
slice_viewer.slicer_view()

interactive(children=(Dropdown(description='slice_view', options=('z', 't', 'x', 'y'), value='z'), IntSlider(v…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [10]:
f.close()

## Unseen tasks --- Motion Artifact simulation M&M

In [8]:
src_path = "/home/arun/datasets/M_M/Training/Labeled/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/unseen_artifact_suppression/"

degradation_type = "MotionM" # Change according to the artifact to be simulated

no_of_train_spt = 37 # 15 for M/04, 22 for M/06 --- all tasks train support data should have 700 slices each
no_of_train_qry = 37 # 12 for M/04, 18 for M/06 --- all tasks train query data should have 700 slices each

no_of_val_spt = 37 # 11 for M/04, 23 for M/06 --- all tasks validation support data should have 700 slices each
no_of_val_qry = 37 # 16 for M/04, 23 for M/06 --- all tasks validation query data should have 700 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [10]:
no_of_frame_mixes = [4,6]
for frame_mix_number in no_of_frame_mixes:
    for number_of_slices_key,i in tqdm(zip(unseen_fixed_no_of_slices,dict_file_names)):

        folder_bits = i.split("_")
        folder_cat = degradation_type+"/"+"0{}/".format(frame_mix_number)+folder_bits[0]+"_"+folder_bits[1]+"/"
        clear_dst_dir = dst_dir+folder_cat

        if not os.path.exists(clear_dst_dir):
            os.makedirs(clear_dst_dir)

        specific_folder_file_names = dict_file_names[i]

        count_slices = 0
        for file_name in specific_folder_file_names:

            with h5py.File(file_name,'r') as hf1:

                img_vol = hf1['inpVol']

                x_center = hf1["x_center"][0]
                y_center = hf1["y_center"][0]

                norm_img_vol = normalize_4d_volume(img_vol)

                clean_4d_vol,corrupted_4d_vol = Motion_simulation(norm_img_vol,frame_mix_number)

                x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
                if np.min(x_coords) < 0:
                    x_coords = x_coords-np.min(x_coords)

                y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
                if np.min(y_coords) < 0:
                    y_coords = y_coords-np.min(y_coords)

                corrupted_cropped_4d_vol = corrupted_4d_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                cropped_gt_vol = clean_4d_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

                patient_name = file_name.split("/")[-1].split("_")[0]
                file_final_destination = clear_dst_dir+patient_name+".h5"

                with h5py.File(file_final_destination,'w') as hf2:

                    hf2.create_dataset('gt',data = clean_4d_vol)

                    hf2.create_dataset("inp",data = corrupted_4d_vol)

                    hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                    hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                    hf2.create_dataset("x_center",data = [x_center])

                    hf2.create_dataset("y_center",data = [y_center])

            if count_slices >= unseen_fixed_no_of_slices[number_of_slices_key]:
                break


0it [00:00, ?it/s]/home/arun/dataset_creation/Artifact_simulation.py:37: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  ksp_line_seq = np.hstack(([linenum for linenum in range(lineblock,rows, n_by_etl)] for lineblock in range(0,n_by_etl)))
4it [09:40, 145.10s/it]
4it [12:36, 189.13s/it]


In [ ]:
disp_file_name = glob.glob("/media/Data/MRI/datasets/unseen_artifact_suppression/MotionM/04/train_support/*.h5")[0]

f = h5py.File(disp_file_name,'r')

slice_viewer = slicer(f)
slice_viewer.slicer_view()

In [ ]:
f.close()

## Training tasks --- Spatial Artifact simulation M&M

In [75]:
src_path = "/home/arun/datasets/M_M/Training/Labeled/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/MM_artifact_suppression/"

degradation_type = "SpatialM" # Change according to the artifact to be simulated

no_of_train_spt = 70 # 9 --- all tasks train support data should have 1500 slices each
no_of_train_qry = 40 # 3 --- all tasks train query data should have 600 slices each

no_of_val_spt = 20 # 3 --- all tasks validation support data should have 450 slices each
no_of_val_qry = 20 # 3 --- all tasks validation query data should have 500 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [ ]:
downsampling_factors = [2,3,5]

for downsampling_factor in downsampling_factors:
    for number_of_slices_key,i in tqdm(zip(seen_fixed_no_of_slices,dict_file_names)):

        folder_bits = i.split("_")
        folder_cat = degradation_type+"/"+"0{}/".format(downsampling_factor)+folder_bits[0]+"_"+folder_bits[1]+"/"
        clear_dst_dir = dst_dir+folder_cat

        if not os.path.exists(clear_dst_dir):
            os.makedirs(clear_dst_dir)

        specific_folder_file_names = dict_file_names[i]

        count_slices = 0
        for file_name in specific_folder_file_names:

            with h5py.File(file_name,'r') as hf1:

                img_vol = hf1['inpVol']

                x_center = hf1["x_center"][0]
                y_center = hf1["y_center"][0]

                norm_img_vol = normalize_4d_volume(img_vol)

                corrupted_4d_vol = Spatial_simulation(norm_img_vol,downsampling_factor)

                x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
                if np.min(x_coords) < 0:
                    x_coords = x_coords-np.min(x_coords)

                y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
                if np.min(y_coords) < 0:
                    y_coords = y_coords-np.min(y_coords)

                corrupted_cropped_4d_vol = corrupted_4d_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                cropped_gt_vol = norm_img_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

                patient_name = file_name.split("/")[-1].split("_")[0]
                file_final_destination = clear_dst_dir+patient_name+".h5"

                with h5py.File(file_final_destination,'w') as hf2:

                    hf2.create_dataset('gt',data = norm_img_vol)

                    hf2.create_dataset("inp",data = corrupted_4d_vol)

                    hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                    hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                    hf2.create_dataset("x_center",data = [x_center])

                    hf2.create_dataset("y_center",data = [y_center])

            if count_slices >= seen_fixed_no_of_slices[number_of_slices_key]:
                break


In [11]:
disp_file_name = glob.glob("/media/Data/MRI/datasets/artifact_suppression/Spatial/05/train_support/*.h5")[0]

f = h5py.File(disp_file_name,'r')

slice_viewer = slicer(f)
slice_viewer.slicer_view()

interactive(children=(Dropdown(description='slice_view', options=('z', 't', 'x', 'y'), value='z'), IntSlider(v…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [18]:
f.close()

## Unseen tasks --- Spatial Artifact simulation M&M

In [3]:
src_path = "/home/arun/datasets/M_M/Training/Labeled/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/unseen_artifact_suppression/"

degradation_type = "SpatialM" # Change according to the artifact to be simulated

no_of_train_spt = 37 # 5 --- all tasks train support data should have 700 slices each
no_of_train_qry = 37 # 4 --- all tasks train query data should have 700 slices each

no_of_val_spt = 37 # 4 --- all tasks validation support data should have 700 slices each
no_of_val_qry = 37 # 4 --- all tasks validation query data should have 700 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [5]:
downsampling_factors = [4]

for downsampling_factor in downsampling_factors:
    for number_of_slices_key,i in tqdm(zip(unseen_fixed_no_of_slices,dict_file_names)):

        folder_bits = i.split("_")
        folder_cat = degradation_type+"/"+"0{}/".format(downsampling_factor)+folder_bits[0]+"_"+folder_bits[1]+"/"
        clear_dst_dir = dst_dir+folder_cat

        if not os.path.exists(clear_dst_dir):
            os.makedirs(clear_dst_dir)

        specific_folder_file_names = dict_file_names[i]

        count_slices = 0
        for file_name in specific_folder_file_names:

            with h5py.File(file_name,'r') as hf1:

                img_vol = hf1['inpVol']

                x_center = hf1["x_center"][0]
                y_center = hf1["y_center"][0]

                norm_img_vol = normalize_4d_volume(img_vol)

                corrupted_4d_vol = Spatial_simulation(norm_img_vol,downsampling_factor)

                x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
                if np.min(x_coords) < 0:
                    x_coords = x_coords-np.min(x_coords)

                y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
                if np.min(y_coords) < 0:
                    y_coords = y_coords-np.min(y_coords)

                corrupted_cropped_4d_vol = corrupted_4d_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                cropped_gt_vol = norm_img_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

                patient_name = file_name.split("/")[-1].split("_")[0]
                file_final_destination = clear_dst_dir+patient_name+".h5"

                with h5py.File(file_final_destination,'w') as hf2:

                    hf2.create_dataset('gt',data = norm_img_vol)

                    hf2.create_dataset("inp",data = corrupted_4d_vol)

                    hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                    hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                    hf2.create_dataset("x_center",data = [x_center])

                    hf2.create_dataset("y_center",data = [y_center])

            if count_slices >= unseen_fixed_no_of_slices[number_of_slices_key]:
                break


4it [00:36,  9.10s/it]


In [7]:
disp_file_name = glob.glob("/media/Data/MRI/datasets/unseen_artifact_suppression/SpatialM/04/train_support/*.h5")[0]

f = h5py.File(disp_file_name,'r')

slice_viewer = slicer(f)
slice_viewer.slicer_view()

interactive(children=(Dropdown(description='slice_view', options=('z', 't', 'x', 'y'), value='z'), IntSlider(v…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [7]:
f.close()

## Training tasks --- Undersampling Artifact simulation M&M

In [5]:
src_path = "/home/arun/datasets/M_M/Training/Labeled/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/MM_artifact_suppression/"

degradation_type = "UndersamplingM" # Change according to the artifact to be simulated

no_of_train_spt = 70 # 9 --- all tasks train support data should have 1500 slices each
no_of_train_qry = 40 # 3 --- all tasks train query data should have 600 slices each

no_of_val_spt = 20 # 3 --- all tasks validation support data should have 450 slices each
no_of_val_qry = 20 # 3 --- all tasks validation query data should have 500 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [9]:
us_factors = [2,4,6]
sample_n = 12 ## number of lines in center 

for us_factor in us_factors:
    for number_of_slices_key,i in tqdm(zip(seen_fixed_no_of_slices,dict_file_names)):

        folder_bits = i.split("_")
        folder_cat = degradation_type+"/"+"0{}/".format(us_factor)+folder_bits[0]+"_"+folder_bits[1]+"/"
        clear_dst_dir = dst_dir+folder_cat
        
        us_mask_path = "/home/arun/datasets/npy_masks/mask_0{}.npy".format(us_factor)
        if not os.path.exists(us_mask_path):
            npy_mask = cartesian_mask(ROI_size,us_factor,sample_n,centred=False)
            np.save(us_mask_path,npy_mask) 

        if not os.path.exists(clear_dst_dir):
            os.makedirs(clear_dst_dir)

        specific_folder_file_names = dict_file_names[i]

        count_slices = 0
        for file_name in specific_folder_file_names:

            with h5py.File(file_name,'r') as hf1:

                img_vol = hf1['inpVol']

                x_center = hf1["x_center"][0]
                y_center = hf1["y_center"][0]

                norm_img_vol = normalize_4d_volume(img_vol)

                x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
                if np.min(x_coords) < 0:
                    x_coords = x_coords-np.min(x_coords)

                y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
                if np.min(y_coords) < 0:
                    y_coords = y_coords-np.min(y_coords)

                cropped_gt_vol = norm_img_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]
                
                height,width,no_of_slices,no_of_frames = cropped_gt_vol.shape
                corrupted_cropped_4d_vol = np.zeros(cropped_gt_vol.shape)
                corrupted_cropped_4d_ksp_vol = np.zeros(cropped_gt_vol.shape)
                for frame_no in range(no_of_frames):

                    corrupted_cropped_4d_vol[:,:,:,frame_no],corrupted_cropped_4d_ksp_vol[:,:,:,frame_no] = Undersampling_simulation(cropped_gt_vol[:,:,:,frame_no],us_factor,us_mask_path)

                count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

                patient_name = file_name.split("/")[-1].split("_")[0]
                file_final_destination = clear_dst_dir+patient_name+".h5"

                with h5py.File(file_final_destination,'w') as hf2:

                    hf2.create_dataset('gt',data = norm_img_vol)

                    # hf2.create_dataset("inp",data = corrupted_4d_vol)

                    hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                    hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                    hf2.create_dataset("crp_ksp",data = corrupted_cropped_4d_ksp_vol)

                    hf2.create_dataset("ksp_mask",data = np.load(us_mask_path))

                    hf2.create_dataset("x_center",data = [x_center])

                    hf2.create_dataset("y_center",data = [y_center])

            if count_slices >= seen_fixed_no_of_slices[number_of_slices_key]:
                break


0it [00:00, ?it/s]/tmp/ipykernel_3044507/2361904475.py:48: ComplexWarning: Casting complex values to real discards the imaginary part
  corrupted_cropped_4d_vol[:,:,:,frame_no],corrupted_cropped_4d_ksp_vol[:,:,:,frame_no] = Undersampling_simulation(cropped_gt_vol[:,:,:,frame_no],us_factor,us_mask_path)
1it [01:07, 67.89s/it]
1it [01:07, 67.65s/it]
1it [01:07, 67.71s/it]


In [1]:
disp_file_name = glob.glob("/media/Data/MRI/datasets/MM_artifact_suppression/UndersamplingM/02/valid_query/*.h5")[0]

f = h5py.File(disp_file_name,'r')

slice_viewer = slicer(f)
slice_viewer.slicer_view()

In [8]:
f.close()

## Unseen tasks --- Undersampling Artifact simulation M&M

In [3]:
src_path = "/home/arun/datasets/M_M/Training/Labeled/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/unseen_artifact_suppression/"

degradation_type = "UndersamplingM" # Change according to the artifact to be simulated

no_of_train_spt = 37 # 5 --- all tasks train support data should have 700 slices each
no_of_train_qry = 37 # 4 --- all tasks train query data should have 700 slices each

no_of_val_spt = 37 # 4 --- all tasks validation support data should have 700 slices each
no_of_val_qry = 37 # 4 --- all tasks validation query data should have 700 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [5]:
us_factors = [3,5]
sample_n = 12 ## number of lines in center 

for us_factor in us_factors:
    for number_of_slices_key,i in tqdm(zip(unseen_fixed_no_of_slices,dict_file_names)):

        folder_bits = i.split("_")
        folder_cat = degradation_type+"/"+"0{}/".format(us_factor)+folder_bits[0]+"_"+folder_bits[1]+"/"
        clear_dst_dir = dst_dir+folder_cat
        
        us_mask_path = "/home/arun/datasets/npy_masks/mask_0{}.npy".format(us_factor)
        if not os.path.exists(us_mask_path):
            npy_mask = cartesian_mask(ROI_size,us_factor,sample_n,centred=False)
            np.save(us_mask_path,npy_mask) 

        if not os.path.exists(clear_dst_dir):
            os.makedirs(clear_dst_dir)

        specific_folder_file_names = dict_file_names[i]

        count_slices = 0
        for file_name in specific_folder_file_names:

            with h5py.File(file_name,'r') as hf1:

                img_vol = hf1['inpVol']

                x_center = hf1["x_center"][0]
                y_center = hf1["y_center"][0]

                norm_img_vol = normalize_4d_volume(img_vol)

                x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
                if np.min(x_coords) < 0:
                    x_coords = x_coords-np.min(x_coords)

                y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
                if np.min(y_coords) < 0:
                    y_coords = y_coords-np.min(y_coords)

                cropped_gt_vol = norm_img_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]
                
                height,width,no_of_slices,no_of_frames = cropped_gt_vol.shape
                corrupted_cropped_4d_vol = np.zeros(cropped_gt_vol.shape)
                corrupted_cropped_4d_ksp_vol = np.zeros(cropped_gt_vol.shape)
                for frame_no in range(no_of_frames):

                    corrupted_cropped_4d_vol[:,:,:,frame_no],corrupted_cropped_4d_ksp_vol[:,:,:,frame_no] = Undersampling_simulation(cropped_gt_vol[:,:,:,frame_no],us_factor,us_mask_path)

                count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

                patient_name = file_name.split("/")[-1].split("_")[0]
                file_final_destination = clear_dst_dir+patient_name+".h5"

                with h5py.File(file_final_destination,'w') as hf2:

                    hf2.create_dataset('gt',data = norm_img_vol)

                    # hf2.create_dataset("inp",data = corrupted_4d_vol)

                    hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                    hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                    hf2.create_dataset("crp_ksp",data = corrupted_cropped_4d_ksp_vol)

                    hf2.create_dataset("ksp_mask",data = np.load(us_mask_path))

                    hf2.create_dataset("x_center",data = [x_center])

                    hf2.create_dataset("y_center",data = [y_center])

            if count_slices >= unseen_fixed_no_of_slices[number_of_slices_key]:
                break


0it [00:00, ?it/s]/tmp/ipykernel_3031918/3714984296.py:48: ComplexWarning: Casting complex values to real discards the imaginary part
  corrupted_cropped_4d_vol[:,:,:,frame_no],corrupted_cropped_4d_ksp_vol[:,:,:,frame_no] = Undersampling_simulation(cropped_gt_vol[:,:,:,frame_no],us_factor,us_mask_path)
4it [00:35,  8.85s/it]
4it [00:35,  8.80s/it]


In [ ]:
disp_file_name = glob.glob("/media/Data/MRI/datasets/artifact_suppression/UndersamplingM/08/train_support/*.h5")[0]

f = h5py.File(disp_file_name,'r')

slice_viewer = slicer(f)
slice_viewer.slicer_view()

In [ ]:
f.close()

## Unseen tasks --- Ghosting Artifact simulation M&M

In [3]:
src_path = "/home/arun/datasets/M_M/Training/Labeled/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/unseen_artifact_suppression/"

degradation_type = "GhostingM" # Change according to the artifact to be simulated

no_of_train_spt = 37 # 5 --- all tasks train support data should have 700 slices each
no_of_train_qry = 37 # 4 --- all tasks train query data should have 700 slices each

no_of_val_spt = 37 # 4 --- all tasks validation support data should have 700 slices each
no_of_val_qry = 37 # 4 --- all tasks validation query data should have 700 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [4]:
num_ghosts = [4,6]

for num_ghost in num_ghosts:
    for number_of_slices_key,i in tqdm(zip(unseen_fixed_no_of_slices,dict_file_names)):

        folder_bits = i.split("_")
        folder_cat = degradation_type+"/"+"0{}/".format(num_ghost)+folder_bits[0]+"_"+folder_bits[1]+"/"
        clear_dst_dir = dst_dir+folder_cat

        if not os.path.exists(clear_dst_dir):
            os.makedirs(clear_dst_dir)

        specific_folder_file_names = dict_file_names[i]

        count_slices = 0
        for file_name in specific_folder_file_names:

            with h5py.File(file_name,'r') as hf1:

                img_vol = hf1['inpVol']

                x_center = hf1["x_center"][0]
                y_center = hf1["y_center"][0]

                norm_img_vol = normalize_4d_volume(img_vol)

                # corrupted_4d_vol = Ghosting_simulation(norm_img_vol,num_ghost)

                x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
                if np.min(x_coords) < 0:
                    x_coords = x_coords-np.min(x_coords)

                y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
                if np.min(y_coords) < 0:
                    y_coords = y_coords-np.min(y_coords)

                # corrupted_cropped_4d_vol = corrupted_4d_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                cropped_gt_vol = norm_img_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                corrupted_cropped_4d_vol = Ghosting_simulation(cropped_gt_vol,num_ghost)
                corrupted_cropped_4d_vol = np.float64(corrupted_cropped_4d_vol)

                count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

                patient_name = file_name.split("/")[-1].split("_")[0]
                file_final_destination = clear_dst_dir+patient_name+".h5"

                with h5py.File(file_final_destination,'w') as hf2:

                    hf2.create_dataset('gt',data = norm_img_vol)

                    # hf2.create_dataset("inp",data = corrupted_4d_vol)

                    hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                    hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                    hf2.create_dataset("x_center",data = [x_center])

                    hf2.create_dataset("y_center",data = [y_center])

            if count_slices >= unseen_fixed_no_of_slices[number_of_slices_key]:
                break


4it [02:21, 35.30s/it]
4it [02:19, 34.99s/it]


## Unseen tasks --- Spike Artifact simulation M&M

In [3]:
src_path = "/home/arun/datasets/M_M/Training/Labeled/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/unseen_artifact_suppression/"

degradation_type = "SpikingM" # Change according to the artifact to be simulated

no_of_train_spt = 37 # 5 --- all tasks train support data should have 700 slices each
no_of_train_qry = 37 # 4 --- all tasks train query data should have 700 slices each

no_of_val_spt = 37 # 4 --- all tasks validation support data should have 700 slices each
no_of_val_qry = 37 # 4 --- all tasks validation query data should have 700 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [4]:
num_spikes = [3,5]

for num_spike in num_spikes:
    for number_of_slices_key,i in tqdm(zip(unseen_fixed_no_of_slices,dict_file_names)):

        folder_bits = i.split("_")
        folder_cat = degradation_type+"/"+"0{}/".format(num_spike)+folder_bits[0]+"_"+folder_bits[1]+"/"
        clear_dst_dir = dst_dir+folder_cat

        if not os.path.exists(clear_dst_dir):
            os.makedirs(clear_dst_dir)

        specific_folder_file_names = dict_file_names[i]

        count_slices = 0
        for file_name in specific_folder_file_names:

            with h5py.File(file_name,'r') as hf1:

                img_vol = hf1['inpVol']

                x_center = hf1["x_center"][0]
                y_center = hf1["y_center"][0]

                norm_img_vol = normalize_4d_volume(img_vol)

                # corrupted_4d_vol = Spike_simulation(norm_img_vol,num_spike)

                x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
                if np.min(x_coords) < 0:
                    x_coords = x_coords-np.min(x_coords)

                y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
                if np.min(y_coords) < 0:
                    y_coords = y_coords-np.min(y_coords)

                # corrupted_cropped_4d_vol = corrupted_4d_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                cropped_gt_vol = norm_img_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                corrupted_cropped_4d_vol = Spike_simulation(cropped_gt_vol,num_spike)
                corrupted_cropped_4d_vol = np.float64(corrupted_cropped_4d_vol)

                count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

                patient_name = file_name.split("/")[-1].split("_")[0]
                file_final_destination = clear_dst_dir+patient_name+".h5"

                with h5py.File(file_final_destination,'w') as hf2:

                    hf2.create_dataset('gt',data = norm_img_vol)

                    # hf2.create_dataset("inp",data = corrupted_4d_vol)

                    hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                    hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                    hf2.create_dataset("x_center",data = [x_center])

                    hf2.create_dataset("y_center",data = [y_center])

            if count_slices >= unseen_fixed_no_of_slices[number_of_slices_key]:
                break


4it [02:19, 34.94s/it]
4it [02:19, 34.99s/it]


## Unseen tasks --- Noise Artifact simulation M&M

In [3]:
src_path = "/home/arun/datasets/M_M/Training/Labeled/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/unseen_artifact_suppression/"

degradation_type = "NoiseM" # Change according to the artifact to be simulated

no_of_train_spt = 37 # 5 --- all tasks train support data should have 700 slices each
no_of_train_qry = 37 # 4 --- all tasks train query data should have 700 slices each

no_of_val_spt = 37 # 4 --- all tasks validation support data should have 700 slices each
no_of_val_qry = 37 # 4 --- all tasks validation query data should have 700 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [4]:
stds = [3,4]

for std in stds:
    for number_of_slices_key,i in tqdm(zip(unseen_fixed_no_of_slices,dict_file_names)):

        folder_bits = i.split("_")
        folder_cat = degradation_type+"/"+"0{}/".format(std)+folder_bits[0]+"_"+folder_bits[1]+"/"
        clear_dst_dir = dst_dir+folder_cat

        if not os.path.exists(clear_dst_dir):
            os.makedirs(clear_dst_dir)

        specific_folder_file_names = dict_file_names[i]

        count_slices = 0
        for file_name in specific_folder_file_names:

            with h5py.File(file_name,'r') as hf1:

                img_vol = hf1['inpVol']

                x_center = hf1["x_center"][0]
                y_center = hf1["y_center"][0]

                norm_img_vol = normalize_4d_volume(img_vol)

                # corrupted_4d_vol = Noise_simulation(norm_img_vol,std = std/100)

                x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
                if np.min(x_coords) < 0:
                    x_coords = x_coords-np.min(x_coords)

                y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
                if np.min(y_coords) < 0:
                    y_coords = y_coords-np.min(y_coords)

                # corrupted_cropped_4d_vol = corrupted_4d_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                cropped_gt_vol = norm_img_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                corrupted_cropped_4d_vol = Noise_simulation(cropped_gt_vol,std = std/100)
                corrupted_cropped_4d_vol = np.float64(corrupted_cropped_4d_vol)

                count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

                patient_name = file_name.split("/")[-1].split("_")[0]
                file_final_destination = clear_dst_dir+patient_name+".h5"

                with h5py.File(file_final_destination,'w') as hf2:

                    hf2.create_dataset('gt',data = norm_img_vol)

                    # hf2.create_dataset("inp",data = corrupted_4d_vol)

                    hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                    hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                    hf2.create_dataset("x_center",data = [x_center])

                    hf2.create_dataset("y_center",data = [y_center])

            if count_slices >= unseen_fixed_no_of_slices[number_of_slices_key]:
                break


4it [02:19, 34.90s/it]
4it [02:19, 34.95s/it]


## Unseen tasks --- Gamma Artifact simulation M&M

In [3]:
src_path = "/home/arun/datasets/M_M/Training/Labeled/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/unseen_artifact_suppression/"

degradation_type = "GammaM" # Change according to the artifact to be simulated

no_of_train_spt = 37 # 5 --- all tasks train support data should have 700 slices each
no_of_train_qry = 37 # 4 --- all tasks train query data should have 700 slices each

no_of_val_spt = 37 # 4 --- all tasks validation support data should have 700 slices each
no_of_val_qry = 37 # 4 --- all tasks validation query data should have 700 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [4]:
gammas = [1,2]

for gamma in gammas:
    for number_of_slices_key,i in tqdm(zip(unseen_fixed_no_of_slices,dict_file_names)):

        folder_bits = i.split("_")
        folder_cat = degradation_type+"/"+"0{}/".format(gamma)+folder_bits[0]+"_"+folder_bits[1]+"/"
        clear_dst_dir = dst_dir+folder_cat

        if not os.path.exists(clear_dst_dir):
            os.makedirs(clear_dst_dir)

        specific_folder_file_names = dict_file_names[i]

        count_slices = 0
        for file_name in specific_folder_file_names:

            with h5py.File(file_name,'r') as hf1:

                img_vol = hf1['inpVol']

                x_center = hf1["x_center"][0]
                y_center = hf1["y_center"][0]

                norm_img_vol = normalize_4d_volume(img_vol)

                # corrupted_4d_vol = Gamma_simulation(norm_img_vol,-gamma/10)
                # corrupted_4d_vol = np.float64(corrupted_4d_vol)

                x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
                if np.min(x_coords) < 0:
                    x_coords = x_coords-np.min(x_coords)

                y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
                if np.min(y_coords) < 0:
                    y_coords = y_coords-np.min(y_coords)

                # corrupted_cropped_4d_vol = corrupted_4d_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                cropped_gt_vol = norm_img_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

                corrupted_cropped_4d_vol = Gamma_simulation(cropped_gt_vol,-gamma/10)
                corrupted_cropped_4d_vol = np.float64(corrupted_cropped_4d_vol)

                count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

                patient_name = file_name.split("/")[-1].split("_")[0]
                file_final_destination = clear_dst_dir+patient_name+".h5"

                with h5py.File(file_final_destination,'w') as hf2:

                    hf2.create_dataset('gt',data = norm_img_vol)

                    # hf2.create_dataset("inp",data = corrupted_4d_vol)

                    hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                    hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                    hf2.create_dataset("x_center",data = [x_center])

                    hf2.create_dataset("y_center",data = [y_center])

            if count_slices >= unseen_fixed_no_of_slices[number_of_slices_key]:
                break


4it [02:20, 35.06s/it]
4it [02:22, 35.67s/it]


# M&M Composite artifacts

## Undersampling and spiking --- M&M

In [3]:
src_path = "/home/arun/datasets/M_M/Training/Labeled/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/unseen_comp_artifact_suppression/"

degradation_types = ["UndersamplingM","SpikingM"] # Only two types. Change according to the artifact to be simulated

no_of_train_spt = 37 # 5 --- all tasks train support data should have 700 slices each
no_of_train_qry = 37 # 4 --- all tasks train query data should have 700 slices each

no_of_val_spt = 37 # 4 --- all tasks validation support data should have 700 slices each
no_of_val_qry = 37 # 4 --- all tasks validation query data should have 700 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [4]:
degradation_amounts= [3,3] # Only two numbers

for number_of_slices_key,i in tqdm(zip(unseen_fixed_no_of_slices,dict_file_names)):

    folder_bits = i.split("_")
    folder_cat = degradation_types[0]+"_"+degradation_types[1]+"/"+"{}_{}/".format(degradation_amounts[0],degradation_amounts[1])+folder_bits[0]+"_"+folder_bits[1]+"/"
    clear_dst_dir = dst_dir+folder_cat

    if not os.path.exists(clear_dst_dir):
        os.makedirs(clear_dst_dir)

    us_factor = degradation_amounts[0]
    num_spike = degradation_amounts[1]

    sample_n = 12 ## number of lines in center
    us_mask_path = "/home/arun/datasets/npy_masks/mask_0{}.npy".format(us_factor)
    if not os.path.exists(us_mask_path):
        npy_mask = cartesian_mask(ROI_size,us_factor,sample_n,centred=False)
        np.save(us_mask_path,npy_mask) 

    specific_folder_file_names = dict_file_names[i]

    count_slices = 0
    for file_name in specific_folder_file_names:

        with h5py.File(file_name,'r') as hf1:

            img_vol = hf1['inpVol']

            x_center = hf1["x_center"][0]
            y_center = hf1["y_center"][0]

            norm_img_vol = normalize_4d_volume(img_vol)

            x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
            if np.min(x_coords) < 0:
                x_coords = x_coords-np.min(x_coords)

            y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
            if np.min(y_coords) < 0:
                y_coords = y_coords-np.min(y_coords)

            cropped_gt_vol = norm_img_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

            height,width,no_of_slices,no_of_frames = cropped_gt_vol.shape
            us_corrupted_cropped_4d_vol = np.zeros(cropped_gt_vol.shape)
            corrupted_cropped_4d_ksp_vol = np.zeros(cropped_gt_vol.shape)
            for frame_no in range(no_of_frames):

                us_corrupted_cropped_4d_vol[:,:,:,frame_no],corrupted_cropped_4d_ksp_vol[:,:,:,frame_no] = Undersampling_simulation(cropped_gt_vol[:,:,:,frame_no],us_factor,us_mask_path)

            corrupted_cropped_4d_vol = Spike_simulation(us_corrupted_cropped_4d_vol,num_spike)
            corrupted_cropped_4d_vol = np.float64(corrupted_cropped_4d_vol)

            count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

            patient_name = file_name.split("/")[-1].split("_")[0]
            file_final_destination = clear_dst_dir+patient_name+".h5"

            with h5py.File(file_final_destination,'w') as hf2:

                hf2.create_dataset('gt',data = norm_img_vol)

                # hf2.create_dataset("inp",data = corrupted_4d_vol)

                hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                hf2.create_dataset("crp_ksp",data = corrupted_cropped_4d_ksp_vol)

                hf2.create_dataset("ksp_mask",data = np.load(us_mask_path))

                hf2.create_dataset("x_center",data = [x_center])

                hf2.create_dataset("y_center",data = [y_center])

        if count_slices >= unseen_fixed_no_of_slices[number_of_slices_key]:
            break


0it [00:00, ?it/s]/tmp/ipykernel_850459/1970259597.py:50: ComplexWarning: Casting complex values to real discards the imaginary part
  us_corrupted_cropped_4d_vol[:,:,:,frame_no],corrupted_cropped_4d_ksp_vol[:,:,:,frame_no] = Undersampling_simulation(cropped_gt_vol[:,:,:,frame_no],us_factor,us_mask_path)
4it [03:02, 45.72s/it]


## Noise and spiking --- M&M

In [3]:
src_path = "/home/arun/datasets/M_M/Training/Labeled/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/unseen_comp_artifact_suppression/"

degradation_types = ["NoiseM","SpikingM"] # Only two types. Change according to the artifact to be simulated

no_of_train_spt = 37 # 5 --- all tasks train support data should have 700 slices each
no_of_train_qry = 37 # 4 --- all tasks train query data should have 700 slices each

no_of_val_spt = 37 # 4 --- all tasks validation support data should have 700 slices each
no_of_val_qry = 37 # 4 --- all tasks validation query data should have 700 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [4]:
degradation_amounts= [3,3] # Only two numbers

for number_of_slices_key,i in tqdm(zip(unseen_fixed_no_of_slices,dict_file_names)):

    folder_bits = i.split("_")
    folder_cat = degradation_types[0]+"_"+degradation_types[1]+"/"+"{}_{}/".format(degradation_amounts[0],degradation_amounts[1])+folder_bits[0]+"_"+folder_bits[1]+"/"
    clear_dst_dir = dst_dir+folder_cat

    if not os.path.exists(clear_dst_dir):
        os.makedirs(clear_dst_dir)

    std = degradation_amounts[0]
    num_spike = degradation_amounts[1]

    specific_folder_file_names = dict_file_names[i]

    count_slices = 0
    for file_name in specific_folder_file_names:

        with h5py.File(file_name,'r') as hf1:

            img_vol = hf1['inpVol']

            x_center = hf1["x_center"][0]
            y_center = hf1["y_center"][0]

            norm_img_vol = normalize_4d_volume(img_vol)

            x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
            if np.min(x_coords) < 0:
                x_coords = x_coords-np.min(x_coords)

            y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
            if np.min(y_coords) < 0:
                y_coords = y_coords-np.min(y_coords)

            cropped_gt_vol = norm_img_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

            noise_corrupted_cropped_4d_vol = Noise_simulation(cropped_gt_vol,std = std/100)

            corrupted_cropped_4d_vol = Spike_simulation(noise_corrupted_cropped_4d_vol,num_spike)
            corrupted_cropped_4d_vol = np.float64(corrupted_cropped_4d_vol)

            count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

            patient_name = file_name.split("/")[-1].split("_")[0]
            file_final_destination = clear_dst_dir+patient_name+".h5"

            with h5py.File(file_final_destination,'w') as hf2:

                hf2.create_dataset('gt',data = norm_img_vol)

                # hf2.create_dataset("inp",data = corrupted_4d_vol)

                hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                hf2.create_dataset("x_center",data = [x_center])

                hf2.create_dataset("y_center",data = [y_center])

        if count_slices >= unseen_fixed_no_of_slices[number_of_slices_key]:
            break


4it [03:00, 45.22s/it]


## Undersampling and noise --- M&M

In [3]:
src_path = "/home/arun/datasets/M_M/Training/Labeled/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/unseen_comp_artifact_suppression/"

degradation_types = ["UndersamplingM","NoiseM"] # Only two types. Change according to the artifact to be simulated

no_of_train_spt = 37 # 5 --- all tasks train support data should have 700 slices each
no_of_train_qry = 37 # 4 --- all tasks train query data should have 700 slices each

no_of_val_spt = 37 # 4 --- all tasks validation support data should have 700 slices each
no_of_val_qry = 37 # 4 --- all tasks validation query data should have 700 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [ ]:
degradation_amounts= [3,3] # Only two numbers

for number_of_slices_key,i in tqdm(zip(unseen_fixed_no_of_slices,dict_file_names)):

    folder_bits = i.split("_")
    folder_cat = degradation_types[0]+"_"+degradation_types[1]+"/"+"{}_{}/".format(degradation_amounts[0],degradation_amounts[1])+folder_bits[0]+"_"+folder_bits[1]+"/"
    clear_dst_dir = dst_dir+folder_cat

    if not os.path.exists(clear_dst_dir):
        os.makedirs(clear_dst_dir)

    us_factor = degradation_amounts[0]
    std = degradation_amounts[1]

    sample_n = 12 ## number of lines in center
    us_mask_path = "/home/arun/datasets/npy_masks/mask_0{}.npy".format(us_factor)
    if not os.path.exists(us_mask_path):
        npy_mask = cartesian_mask(ROI_size,us_factor,sample_n,centred=False)
        np.save(us_mask_path,npy_mask) 

    specific_folder_file_names = dict_file_names[i]

    count_slices = 0
    for file_name in specific_folder_file_names:

        with h5py.File(file_name,'r') as hf1:

            img_vol = hf1['inpVol']

            x_center = hf1["x_center"][0]
            y_center = hf1["y_center"][0]

            norm_img_vol = normalize_4d_volume(img_vol)

            x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
            if np.min(x_coords) < 0:
                x_coords = x_coords-np.min(x_coords)

            y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
            if np.min(y_coords) < 0:
                y_coords = y_coords-np.min(y_coords)

            cropped_gt_vol = norm_img_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

            height,width,no_of_slices,no_of_frames = cropped_gt_vol.shape
            us_corrupted_cropped_4d_vol = np.zeros(cropped_gt_vol.shape)
            corrupted_cropped_4d_ksp_vol = np.zeros(cropped_gt_vol.shape)
            for frame_no in range(no_of_frames):

                us_corrupted_cropped_4d_vol[:,:,:,frame_no],corrupted_cropped_4d_ksp_vol[:,:,:,frame_no] = Undersampling_simulation(cropped_gt_vol[:,:,:,frame_no],us_factor,us_mask_path)

            corrupted_cropped_4d_vol = Noise_simulation(us_corrupted_cropped_4d_vol,std = std/100)
            corrupted_cropped_4d_vol = np.float64(corrupted_cropped_4d_vol)

            count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

            patient_name = file_name.split("/")[-1].split("_")[0]
            file_final_destination = clear_dst_dir+patient_name+".h5"

            with h5py.File(file_final_destination,'w') as hf2:

                hf2.create_dataset('gt',data = norm_img_vol)

                # hf2.create_dataset("inp",data = corrupted_4d_vol)

                hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                hf2.create_dataset("crp_ksp",data = corrupted_cropped_4d_ksp_vol)

                hf2.create_dataset("ksp_mask",data = np.load(us_mask_path))

                hf2.create_dataset("x_center",data = [x_center])

                hf2.create_dataset("y_center",data = [y_center])

        if count_slices >= unseen_fixed_no_of_slices[number_of_slices_key]:
            break


0it [00:00, ?it/s]/tmp/ipykernel_2739227/4008244272.py:50: ComplexWarning: Casting complex values to real discards the imaginary part
  us_corrupted_cropped_4d_vol[:,:,:,frame_no],corrupted_cropped_4d_ksp_vol[:,:,:,frame_no] = Undersampling_simulation(cropped_gt_vol[:,:,:,frame_no],us_factor,us_mask_path)
3it [01:53, 39.71s/it]

## Undersampling and ghosting --- M&M

In [3]:
src_path = "/home/arun/datasets/M_M/Training/Labeled/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/unseen_comp_artifact_suppression/"

degradation_types = ["UndersamplingM","GhostingM"] # Only two types. Change according to the artifact to be simulated

no_of_train_spt = 37 # 5 --- all tasks train support data should have 700 slices each
no_of_train_qry = 37 # 4 --- all tasks train query data should have 700 slices each

no_of_val_spt = 37 # 4 --- all tasks validation support data should have 700 slices each
no_of_val_qry = 37 # 4 --- all tasks validation query data should have 700 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [4]:
degradation_amounts= [3,3] # Only two numbers

for number_of_slices_key,i in tqdm(zip(unseen_fixed_no_of_slices,dict_file_names)):

    folder_bits = i.split("_")
    folder_cat = degradation_types[0]+"_"+degradation_types[1]+"/"+"{}_{}/".format(degradation_amounts[0],degradation_amounts[1])+folder_bits[0]+"_"+folder_bits[1]+"/"
    clear_dst_dir = dst_dir+folder_cat

    if not os.path.exists(clear_dst_dir):
        os.makedirs(clear_dst_dir)

    us_factor = degradation_amounts[0]
    num_ghost = degradation_amounts[1]

    sample_n = 12 ## number of lines in center
    us_mask_path = "/home/arun/datasets/npy_masks/mask_0{}.npy".format(us_factor)
    if not os.path.exists(us_mask_path):
        npy_mask = cartesian_mask(ROI_size,us_factor,sample_n,centred=False)
        np.save(us_mask_path,npy_mask) 

    specific_folder_file_names = dict_file_names[i]

    count_slices = 0
    for file_name in specific_folder_file_names:

        with h5py.File(file_name,'r') as hf1:

            img_vol = hf1['inpVol']

            x_center = hf1["x_center"][0]
            y_center = hf1["y_center"][0]

            norm_img_vol = normalize_4d_volume(img_vol)

            x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
            if np.min(x_coords) < 0:
                x_coords = x_coords-np.min(x_coords)

            y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
            if np.min(y_coords) < 0:
                y_coords = y_coords-np.min(y_coords)

            cropped_gt_vol = norm_img_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

            height,width,no_of_slices,no_of_frames = cropped_gt_vol.shape
            us_corrupted_cropped_4d_vol = np.zeros(cropped_gt_vol.shape)
            corrupted_cropped_4d_ksp_vol = np.zeros(cropped_gt_vol.shape)
            for frame_no in range(no_of_frames):

                us_corrupted_cropped_4d_vol[:,:,:,frame_no],corrupted_cropped_4d_ksp_vol[:,:,:,frame_no] = Undersampling_simulation(cropped_gt_vol[:,:,:,frame_no],us_factor,us_mask_path)

            corrupted_cropped_4d_vol = Ghosting_simulation(us_corrupted_cropped_4d_vol,num_ghost)
            corrupted_cropped_4d_vol = np.float64(corrupted_cropped_4d_vol)

            count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

            patient_name = file_name.split("/")[-1].split("_")[0]
            file_final_destination = clear_dst_dir+patient_name+".h5"

            with h5py.File(file_final_destination,'w') as hf2:

                hf2.create_dataset('gt',data = norm_img_vol)

                # hf2.create_dataset("inp",data = corrupted_4d_vol)

                hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                hf2.create_dataset("crp_ksp",data = corrupted_cropped_4d_ksp_vol)

                hf2.create_dataset("ksp_mask",data = np.load(us_mask_path))

                hf2.create_dataset("x_center",data = [x_center])

                hf2.create_dataset("y_center",data = [y_center])

        if count_slices >= unseen_fixed_no_of_slices[number_of_slices_key]:
            break


0it [00:00, ?it/s]/tmp/ipykernel_2836297/1885899305.py:50: ComplexWarning: Casting complex values to real discards the imaginary part
  us_corrupted_cropped_4d_vol[:,:,:,frame_no],corrupted_cropped_4d_ksp_vol[:,:,:,frame_no] = Undersampling_simulation(cropped_gt_vol[:,:,:,frame_no],us_factor,us_mask_path)
4it [02:47, 41.87s/it]


## Ghosting and spiking --- M&M

In [3]:
src_path = "/home/arun/datasets/M_M/Training/Labeled/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/unseen_comp_artifact_suppression/"

degradation_types = ["GhostingM","SpikingM"] # Only two types. Change according to the artifact to be simulated

no_of_train_spt = 37 # 5 --- all tasks train support data should have 700 slices each
no_of_train_qry = 37 # 4 --- all tasks train query data should have 700 slices each

no_of_val_spt = 37 # 4 --- all tasks validation support data should have 700 slices each
no_of_val_qry = 37 # 4 --- all tasks validation query data should have 700 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [4]:
degradation_amounts= [3,3] # Only two numbers

for number_of_slices_key,i in tqdm(zip(unseen_fixed_no_of_slices,dict_file_names)):

    folder_bits = i.split("_")
    folder_cat = degradation_types[0]+"_"+degradation_types[1]+"/"+"{}_{}/".format(degradation_amounts[0],degradation_amounts[1])+folder_bits[0]+"_"+folder_bits[1]+"/"
    clear_dst_dir = dst_dir+folder_cat

    if not os.path.exists(clear_dst_dir):
        os.makedirs(clear_dst_dir)

    num_ghost = degradation_amounts[0]
    num_spike = degradation_amounts[1]

    specific_folder_file_names = dict_file_names[i]

    count_slices = 0
    for file_name in specific_folder_file_names:

        with h5py.File(file_name,'r') as hf1:

            img_vol = hf1['inpVol']

            x_center = hf1["x_center"][0]
            y_center = hf1["y_center"][0]

            norm_img_vol = normalize_4d_volume(img_vol)

            x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
            if np.min(x_coords) < 0:
                x_coords = x_coords-np.min(x_coords)

            y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
            if np.min(y_coords) < 0:
                y_coords = y_coords-np.min(y_coords)

            cropped_gt_vol = norm_img_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

            ghosted_corrupted_cropped_4d_vol = Ghosting_simulation(cropped_gt_vol,num_ghost)

            corrupted_cropped_4d_vol = Spike_simulation(ghosted_corrupted_cropped_4d_vol,num_spike)
            corrupted_cropped_4d_vol = np.float64(corrupted_cropped_4d_vol)

            count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

            patient_name = file_name.split("/")[-1].split("_")[0]
            file_final_destination = clear_dst_dir+patient_name+".h5"

            with h5py.File(file_final_destination,'w') as hf2:

                hf2.create_dataset('gt',data = norm_img_vol)

                # hf2.create_dataset("inp",data = corrupted_4d_vol)

                hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                hf2.create_dataset("x_center",data = [x_center])

                hf2.create_dataset("y_center",data = [y_center])

        if count_slices >= unseen_fixed_no_of_slices[number_of_slices_key]:
            break


4it [02:29, 37.48s/it]


## Spatial and noise --- M&M

In [3]:
src_path = "/home/arun/datasets/M_M/Training/Labeled/h5/*.h5"

dst_dir  = "/media/Data/MRI/datasets/unseen_comp_artifact_suppression/"

degradation_types = ["SpatialM","NoiseM"] # Only two types. Change according to the artifact to be simulated

no_of_train_spt = 37 # 5 --- all tasks train support data should have 700 slices each
no_of_train_qry = 37 # 4 --- all tasks train query data should have 700 slices each

no_of_val_spt = 37 # 4 --- all tasks validation support data should have 700 slices each
no_of_val_qry = 37 # 4 --- all tasks validation query data should have 700 slices each

all_file_names = glob.glob(src_path)
train_spt_file_names = all_file_names[0:no_of_train_spt]
train_qry_file_names = all_file_names[no_of_train_spt:no_of_train_spt+no_of_train_qry]

val_spt_file_names = all_file_names[no_of_train_spt+no_of_train_qry:no_of_train_spt+no_of_train_qry+no_of_val_spt]
val_qry_file_names = all_file_names[no_of_train_spt+no_of_train_qry+no_of_val_spt:no_of_train_spt+no_of_train_qry+no_of_val_spt+no_of_val_qry]

dict_file_names = {}
dict_file_names["train_support_file_names"] = train_spt_file_names
dict_file_names["train_query_file_names"] = train_qry_file_names

dict_file_names["valid_support_file_names"] = val_spt_file_names
dict_file_names["valid_query_file_names"] = val_qry_file_names

In [4]:
degradation_amounts= [2,3] # Only two numbers

for number_of_slices_key,i in tqdm(zip(unseen_fixed_no_of_slices,dict_file_names)):

    folder_bits = i.split("_")
    folder_cat = degradation_types[0]+"_"+degradation_types[1]+"/"+"{}_{}/".format(degradation_amounts[0],degradation_amounts[1])+folder_bits[0]+"_"+folder_bits[1]+"/"
    clear_dst_dir = dst_dir+folder_cat

    if not os.path.exists(clear_dst_dir):
        os.makedirs(clear_dst_dir)

    downsampling_factor = degradation_amounts[0]
    std = degradation_amounts[1]

    specific_folder_file_names = dict_file_names[i]

    count_slices = 0
    for file_name in specific_folder_file_names:

        with h5py.File(file_name,'r') as hf1:

            img_vol = hf1['inpVol']

            x_center = hf1["x_center"][0]
            y_center = hf1["y_center"][0]

            norm_img_vol = normalize_4d_volume(img_vol)

            x_coords = np.arange(x_center-int(ROI_size[0]/2),x_center+int(ROI_size[0]/2),1)
            if np.min(x_coords) < 0:
                x_coords = x_coords-np.min(x_coords)

            y_coords = np.arange(y_center-int(ROI_size[1]/2),y_center+int(ROI_size[1]/2),1)
            if np.min(y_coords) < 0:
                y_coords = y_coords-np.min(y_coords)

            cropped_gt_vol = norm_img_vol[np.min(x_coords):np.max(x_coords)+1,np.min(y_coords):np.max(y_coords)+1,:,:]

            spatial_corrupted_4d_vol = Spatial_simulation(cropped_gt_vol,downsampling_factor)

            corrupted_cropped_4d_vol = Noise_simulation(spatial_corrupted_4d_vol,std = std/100)

            corrupted_cropped_4d_vol = np.float64(corrupted_cropped_4d_vol)

            count_slices = count_slices+corrupted_cropped_4d_vol.shape[2]*corrupted_cropped_4d_vol.shape[3]

            patient_name = file_name.split("/")[-1].split("_")[0]
            file_final_destination = clear_dst_dir+patient_name+".h5"

            with h5py.File(file_final_destination,'w') as hf2:

                hf2.create_dataset('gt',data = norm_img_vol)

                # hf2.create_dataset("inp",data = corrupted_4d_vol)

                hf2.create_dataset("crp_inp",data = corrupted_cropped_4d_vol)

                hf2.create_dataset("crp_gt",data = cropped_gt_vol)

                hf2.create_dataset("x_center",data = [x_center])

                hf2.create_dataset("y_center",data = [y_center])

        if count_slices >= unseen_fixed_no_of_slices[number_of_slices_key]:
            break


4it [02:59, 44.96s/it]
